In [1]:
from dotenv import load_dotenv
import os

or_api_key = os.getenv("OR_API_KEY")

In [2]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=or_api_key,
)

In [3]:

def chat_llm(user_prompt, system_prompt="", model="anthropic/claude-sonnet-4"):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ]
    )
    return completion.choices[0].message.content


In [4]:
def call_llm(messages, model="anthropic/claude-sonnet-4"):
    completion = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return completion.choices[0].message.content

# Methodology

## Value Elicitation

- Select dilemma from dataset
- Chat with the agent
    - Why do you prefer it over the other?
    - Give an example of a related moral situation where you might apply this value
- Extract the value from this conversation. 

## Graph Generation
- Pick two random nodes (how to select nodes?)
- Model generates story of transitioning from Node A to Node B
- User asked if transition is wiser, and if yes, a directed edge is created. Repeat for N edges (can do link prediction? majority voting across jury of models?)
- PageRank usage to rate values


In [5]:
import kagglehub
from kagglehub import KaggleDatasetAdapter


/opt/miniconda3/envs/mge/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# !curl -L -o reddit.zip \
#   https://www.kaggle.com/api/v1/datasets/download/jianloongliew/reddit

In [7]:
# !unzip reddit.zip -d reddit_data

In [8]:
# import sqlite3

# # Connect to the SQLite database
# conn = sqlite3.connect('reddit_data/AmItheAsshole.sqlite')

In [9]:
# import pandas as pd

# cursor = conn.cursor()
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# tables = cursor.fetchall()

# dfs = {}
# for tbl in tables:
#     tbl_name = tbl[0]
#     dfs[tbl_name] = pd.read_sql_query(f"SELECT * FROM {tbl_name}", conn)

In [10]:
# for key, df_ in dfs.items():
#     df_.to_csv(f"{key}.csv", index=False)

In [11]:
import pandas as pd

In [12]:
submissions_df = pd.read_csv("submission.csv")
# comments_df = pd.read_csv("comment.csv")

In [13]:
submissions_df[:5]

,id,submission_id,title,selftext,created_utc,permalink,score
0,1,xt1ksm,AITA Monthly Open Forum Spooktober 2022,#Keep things civil. Rules still apply.\n\n##Th...,1664646465,/r/AmItheAsshole/comments/xt1ksm/aita_monthly_...,592
1,2,yiplwk,AITA for asking my friend to move a picture of...,"\n\nMe (M32) and my wife, Dahlia (F28) lost ou...",1667251988,/r/AmItheAsshole/comments/yiplwk/aita_for_aski...,16582
2,3,yiv572,AITA for asking my husband to stay with me whi...,Throwaway my family knows my account. I'll get...,1667266450,/r/AmItheAsshole/comments/yiv572/aita_for_aski...,4079
3,4,yimgaf,AITA for telling my SIL to stop talking about ...,My (37M) wife (37F) is pregnant with our first...,1667245059,/r/AmItheAsshole/comments/yimgaf/aita_for_tell...,9728
4,5,yin7pf,"AITA for wanting to meet my ""daughter"" after g...",Long story short: in my (40f) twenties I had a...,1667246573,/r/AmItheAsshole/comments/yin7pf/aita_for_want...,6889


In [14]:
import prompts
print(dir(prompts))

['ANALOGOUS_SITUATIONS', 'GIVE_MORAL_STANCE', 'PAST_SITUATIONS', 'STRESS_TEST_SITUATIONS', 'VALUE_CARD_GENERATION', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'value_from_situations', 'value_zeroshot']


In [15]:
from prompts import GIVE_MORAL_STANCE, ANALOGOUS_SITUATIONS, STRESS_TEST_SITUATIONS, VALUE_CARD_GENERATION, value_from_situations, value_zeroshot

In [16]:
dilemmas = []
for i,j in zip(submissions_df["title"], submissions_df["selftext"]):
    dilemmas.append(f"{i}\n\n{j}")

In [17]:
dilemmas = dilemmas[1:]

In [19]:
i = 4
print(dilemmas[i])

AITA for making my roommate replace my garlic that she used?

Last week, I came home and my roommate told me that without asking me, she had used the garlic I had in the fridge. I was bothered that she didn’t bother to ask me when it would take a two minute text exchange. She just expected I’d be okay with her using something I bought because we lived together, and didn’t think of how it could inconvenience me if I needed to use garlic that night. To be fair to her, she said she’d replace it, but didn’t give any timeline on when that would be. 

And some important context: my current roommate knows I had a history of shitty roommates that take advantage of my kindness. I chose her specifically because I didn’t expect she’d be the type to do that. Hence why this stung. 

Anyways, told her I was upset that she used it, and that she needed to replace it tonight. She fought back, but went out and bought some. 

She came back with the garlic cloves separated and peeled instead of a fresh bu

In [20]:
moral_stance = chat_llm(dilemmas[i], system_prompt=GIVE_MORAL_STANCE)

In [21]:
print(moral_stance)

ESH | While your roommate should have asked before using your garlic, your reaction was disproportionate to the offense. Demanding immediate replacement "tonight" and then rejecting her first attempt because you preferred a whole bulb over peeled cloves comes across as overly controlling and nitpicky. A reasonable response would have been to simply ask her to replace it when convenient and accept whatever form of garlic she brought back - it's still garlic.

That said, your roommate also sucks for talking behind your back and mocking you to friends instead of addressing any ongoing issues directly with you. Taking a small conflict and turning it into social drama is immature.

The whole situation escalated unnecessarily from what should have been a minor roommate miscommunication into a bigger conflict that's now affecting your social circle.


In [48]:
def return_situations(dilemma, moral_stance):
    generated_sitations = []
    for prompt in [ANALOGOUS_SITUATIONS, STRESS_TEST_SITUATIONS]:
        messages = [
            {
                "role": "system",
                "content": prompt
            },
            {
                "role": "user",
                "content": dilemma
            },
            {
                "role": "assistant",
                "content": moral_stance
            }
        ]
        response = call_llm(messages)
        generated_sitations.append(response)
    return generated_sitations

In [26]:
from random import shuffle

In [27]:
def zero_shot_value_card_generation(dilemma, moral_stance):
    messages = [
        {
            "role": "system",
            "content": situations_system
        },
        {
            "role": "user",
            "content": situations_text
        }
    ]
    response = call_llm(messages)
    return response

In [28]:
# todo semantic dedup and run experiment

In [29]:
from llm_utils import diverse_sampling

In [30]:
help(diverse_sampling)

Help on function diverse_sampling in module llm_utils.diverse_sampling:

diverse_sampling(
    data_list,
    k,
    num_clusters,
    model_name='sentence-transformers/all-MiniLM-L6-v2'
)
    Selects a diverse subset of items from a list of documents.

    This function first embeds all documents into vector representations using a
    sentence transformer model. It then clusters these embeddings into a specified
    number of clusters using k-means. Finally, it samples a number of documents
    from each cluster using a method that promotes diversity based on cosine
    similarity.

    Args:
        data_list (list[str]): A list of documents to sample from.
        k (int): The total number of samples to choose.
        num_clusters (int): The number of clusters to group the documents into.
        model_name (str): Model to use for embedding the documents. Default is
                          "sentence-transformers/all-MiniLM-L6-v2".

    Returns:
        list[int]: A list of indic

In [31]:
len(dilemmas)

30993

In [32]:
unique_dilemmas = diverse_sampling(dilemmas, 100, 25)

Batches: 100%|██████████| 969/969 [03:52<00:00,  4.17it/s]
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
selected_dilemmas = [dilemmas[i] for i in unique_dilemmas]

In [34]:
selected_dilemmas

["AITA because I am forcing my parents to choose between two options they loathe.\n\nI (F38) have recently gotten a not great medical diagnosis and prognosis.  I am at peace with it. \n\nSo I have sat down with my husband and figured out my will.  Obviously he will be receiving most of the estate.  He has to take care of himself and our kids.  He and I both have decent if not spectacular careers and he likes his work.  \n\nMy oldest brother is a gigantic D-bag. He is an addict and my parents have wasted their lives trying to help him. They spent somuch of their time and money on him they had very little formy sister or myself.  I don't hold this against them. I love my kids and I can understand the urge to help them with their problems.  \n\nThe real issue is that my brother has abandoned my nieces with my parents.  I tried to get custody of them but I am not a member of their church and my parents cut me off when I left home to get a college education instead of staying home and helpi

In [39]:
import random
random.seed(42)

In [40]:
shuffle(selected_dilemmas)

In [41]:
import pickle

with open("selected_dilemmas.pkl", "wb") as f:
    pickle.dump(selected_dilemmas, f)

In [36]:
from tqdm import tqdm

In [37]:
def elicit_values_from_dilemmas(dilemmas_to_process):
    """
    Run value elicitation process on a list of dilemmas.
    
    Args:
        dilemmas (list): List of moral dilemma texts
        num_samples (int, optional): Number of random dilemmas to sample. If None, uses all dilemmas.
    
    Returns:
        list: List of dictionaries containing dilemmas and their elicited values
    """
    
    results = []
    
    for idx, dilemma in tqdm(enumerate(dilemmas_to_process)):
        try:
            # Get moral stance
            moral_stance = chat_llm(dilemma, system_prompt=GIVE_MORAL_STANCE)
            
            # Generate related situations
            situations = return_situations(dilemma, moral_stance)
            all_situations = []
            for situation in situations:
                all_situations.extend(situation.split("\n"))
            all_situations.append(dilemma)
            shuffle(all_situations)
            situations_text = ("\n" + "*" * 50 + "\n").join(all_situations)
            
            # Generate value card
            situations_system = value_from_situations(situations_text)
            
            value_card = chat_llm(situations_system)
            
            results.append({
                'dilemma': dilemma,
                'moral_stance': moral_stance,
                'related_situations': situations,
                'value_card': value_card
            })
            
            print(f"Processed dilemma {idx + 1}/{len(dilemmas_to_process)}")
            
        except Exception as e:
            print(f"Error processing dilemma {idx + 1}: {str(e)}")
            continue
    
    return results

In [49]:
def elicit_values_from_dilemmas_zeroshot(dilemmas_to_process):
    """
    Run value elicitation process on a list of dilemmas.
    
    Args:
        dilemmas (list): List of moral dilemma texts
        num_samples (int, optional): Number of random dilemmas to sample. If None, uses all dilemmas.
    
    Returns:
        list: List of dictionaries containing dilemmas and their elicited values
    """
    
    results = []
    
    for idx, dilemma in tqdm(enumerate(dilemmas_to_process)):
        try:
            # Get moral stance
            moral_stance = chat_llm(dilemma, system_prompt=GIVE_MORAL_STANCE)
            
            # Generate related situations
           
            situations_system = value_zeroshot(dilemma, moral_stance)
            value_card = chat_llm(situations_system)
            
            results.append({
                'dilemma': dilemma,
                'moral_stance': moral_stance,
                'value_card': value_card
            })
            
            print(f"Processed dilemma {idx + 1}/{len(dilemmas_to_process)}")
            
        except Exception as e:
            print(f"Error processing dilemma {idx + 1}: {str(e)}")
            continue
    
    return results

In [55]:
subset = selected_dilemmas[5:10]

In [56]:
values = elicit_values_from_dilemmas_zeroshot(subset)
values_2 = elicit_values_from_dilemmas(subset)

1it [00:11, 11.50s/it]

Processed dilemma 1/5


2it [00:20,  9.99s/it]

Processed dilemma 2/5


3it [00:30,  9.86s/it]

...

Processed dilemma 3/5


4it [01:21, 20.68s/it]

Processed dilemma 4/5


5it [01:42, 20.60s/it]

Processed dilemma 5/5


In [57]:
for v in subset:
    print(v)
    print("\n" + "=" * 50 + "\n")

WIBTA for siding with my mother about a vacation? [29F/34M]

For some context, my husband grew up quite well off and has seen a lot of the world during his childhood and early adulthood. My family could barely even afford a day at the zoo once a year. 

My mother 50F who is disabled and on benefits recently won £10k and wants to go on a once in a lifetime holiday with me and my sister next summer.

My husband is the sole earner as I am a student and disabled, so my future earning potential is also quite low. He makes around £120k per year, sometimes more with bonuses. We have just over £100k savings which we put around £2k into every month. 

The vacations my mother wants to do is a cruise that will go to a few countries in Europe lasting 10 days, and is likely to cost around £2500 all inclusive.

My husband says this is too expensive and has asked that I talk her into doing something smaller and more affordable. I tried, but she is absolutely set on a cruise as it’s always been her dr

In [58]:
for k in values:
    print(k['value_card'])
    print("\n" + "=" * 50 + "\n")

{
  "value_name": "Empathetic Justice and Family Solidarity",
  "description": "Prioritizing the needs and dreams of disadvantaged family members over personal comfort, especially when one has the means to help without significant sacrifice, and recognizing that those with privilege should use their resources to support once-in-a-lifetime opportunities for those who have had fewer advantages",
  "applications": "Family financial decisions, supporting disabled or economically disadvantaged relatives, balancing personal desires against meaningful opportunities for loved ones, and using financial privilege responsibly to create equity within family relationships"
}


{
  "value_name": "Reciprocal Justice",
  "description": "The principle that people should experience the same treatment they give others to understand the impact of their actions and promote fairness",
  "applications": "Addressing patterns of disrespectful behavior, teaching empathy through direct experience, and challengin

In [59]:
for k in values_2:
    print(k['value_card'])
    print("\n" + "=" * 50 + "\n")

{
  "value_name": "Proportional financial sacrifice based on capacity and relationship obligations",
  "description": "The moral weight of financial decisions should be evaluated based on the relative financial impact on the giver, the significance of the need or occasion, and the strength of relational obligations. Those with greater financial capacity bear greater moral responsibility to help with meaningful needs, while the legitimacy of requests depends on their necessity and the appropriateness of alternative options.",
  "applications": "Family medical emergencies, once-in-a-lifetime meaningful experiences for loved ones, distinguishing between reasonable support requests versus ongoing financial enabling, and balancing personal financial boundaries with relational duties based on one's economic position"
}


{
  "value_name": "Reciprocal moral consistency",
  "description": "The principle that one should apply the same moral standards to their own behavior that they expect from 